In [45]:
from pathlib import Path
import random
import os

import clip
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split, Subset
import torchmetrics
import torchvision
from tqdm import tqdm
from PIL import Image

from oma_recipeclassifier.src.model_dev.few_shot.balanced_batch_sampler import BalancedBatchSampler

In [46]:
def finetune():
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)
    
    SAVE_INTERVAL = 9
    BATCH_SIZE = 32
    NUM_EPOCHS = 10

    # GPT - Inconsistencies in handling model precision
    def convert_models_to_fp32(model):
        for p in model.parameters():
            p.data = p.data.float()
            if p.requires_grad():
                p.grad.data = p.grad.data.float()

    # ----------
    # Load model 
    # ----------
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device, jit=False) # jit=False to disable TorchScript for fine-tuning
    # GPT - These lines are unnecessary?
    if device == "cpu":
        model.float() # Converts model to fp32
    else:
        clip.model.convert_weights(model) # Converts model to fp16 (unnecessary since CLIP already uses fp16 by default)
    
    # TensorBoard writer
    writer = SummaryWriter()
    weights_path = Path("model_checkpoints") # Dir to save model checkpoints
    weights_path.mkdir(exist_ok=True)

    # ----------
    # Load dataset 
    # ----------
    dataset = torchvision.datasets.ImageFolder("dataset", transform=preprocess)
    
    # Split dataset into train/test
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_indices, test_indices = random_split(range(len(dataset)), [train_size, test_size])
    train_dataset, test_dataset = Subset(dataset, train_indices), Subset(dataset, test_indices)
    
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    
    # Extract train/test labels
    train_labels = torch.tensor([dataset.targets[i] for i in train_indices])
    test_labels = torch.tensor([dataset.targets[i] for i in test_indices])
    
    # Create DataLoaders
    train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler)
    test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler)
    
    # Display train/test dataset distribution
    
    # ----------
    # Training 
    # ----------
    # Loss functions
    loss_img = torch.nn.CrossEntropyLoss()
    loss_txt = torch.nn.CrossEntropyLoss(ignore_index=-1)
    
    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=1e-7, weight_decay=1e-4)
    
    num_batches_train = len(train_dataloader.dataset) / BATCH_SIZE
    num_batches_test = len(test_dataloader.dataset) / BATCH_SIZE
    
    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")
        epoch_train_loss = 0
        model.train()
        
        for batch in tqdm(train_dataloader, total=num_batches_train):
            optimizer.zero_grad() # Clear gradients from previous iteration
            
            images, label_ids = batch
            
            images = torch.stack([img for img in images], dim=0).to(device) # Stack images into a single tensor (adds an extra dim representing the batch)
            # Generate text prompts: the number of text prompts will be equal to the number of images in the batch (label_ids)
            # Scenario: matching each image with its corresponding text prompt, doesn't allow for comparison against other text prompts
            # texts = [f"A photo of a {train_dataset.dataset.classes[label_id]}" for label_id in label_ids]
            texts = [f"A photo of a {dataset.classes[label_id]}" for label_id in label_ids]
            text = clip.tokenize(texts).to(device) # Tokenize text prompts
            
            print(f"images shape: {images.shape}")
            print(f"label_ids shape: {label_ids.shape}")
            print(f"label_ids: {label_ids}")
            print(f"text shape: {text.shape}")
            
            logits_per_image, logits_per_text = model(images, text) # Forward pass
    
            # Ground truth labels: For each batch, the i-th image corresponds to the i-th text
            # Therefore, the i-th image should have the same label as the i-th text, i.e. [0, 1, 2, ..., BATCH_SIZE - 1]
            # The same happens for text, so we use the same ground truth for both image and text
            ground_truth = torch.arange(logits_per_image.shape[0], dtype=torch.long, device=device)
            
            print(f"logits_per_image shape: {logits_per_image.shape}")
            print(f"logits_per_text shape: {logits_per_text.shape}")
            print(f"ground_truth shape: {ground_truth.shape}")
            print(f"ground_truth: {ground_truth}")
    
            # Compute loss
            total_train_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2
            total_train_loss.backward() # Backward pass
            epoch_train_loss += total_train_loss
            
            torch.nn.utils.clip_grad_norm_(params, 1.0) # Clip gradients to prevent exploding gradients
            
            if device == "cpu":
                optimizer.step() # Update weights
            else:
                convert_models_to_fp32(model)
                optimizer.step() # Update weights
                clip.model.convert_weights(model)
            
        epoch_train_loss /= num_batches_train # Average loss per epoch
        writer.add_scalar("Loss/train", epoch_train_loss, epoch) # Log loss to TensorBoard
        
        # Save model weights
        if epoch % SAVE_INTERVAL == 0:
            torch.save({"epoch": epoch, "model_state_dict": model.state_dict(), "optimizer_state_dict": optimizer.state_dict()}, weights_path / f"epoch_{epoch}.pt")
            print(f"Saved weights under {weights_path}/epoch_{epoch}.pt")
           
        # ----------
        # Testing
        # ----------
        num_batches_test = len(test_dataloader.dataset) / BATCH_SIZE 
        epoch_test_loss = 0
        model.eval()
        
        acc_top3_list = []
        acc_top1_list = []
        
        print("------ Testing ------")
        num_classes = len(dataset.classes)
        classes = torch.arange(num_classes, device=device)
        
        print(f"num_classes: {num_classes}")
        print(f"dataset.classes: {dataset.classes}")
        print(f"classes: {classes}")
        
        for i, batch in enumerate(tqdm(test_dataloader, total=num_batches_test)):
            images, label_ids = batch
            images = images.to(device)
            label_ids = label_ids.to(device)
            
            # Generate text prompts: the number of text prompts will be equal to the number of classes in the dataset (classes)
            # Scenario: classifying each image against all possible classes, allows for comparison against all classes
            texts = torch.cat([clip.tokenize(f"A photo of a {c}") for c in dataset.classes]).to(device) # Concatenate text prompts
            
            print(f"images shape: {images.shape}")
            print(f"label_ids shape: {label_ids.shape}")
            print(f"label_ids: {label_ids}")
            print(f"texts shape: {texts.shape}")

            
            with torch.no_grad():
                image_features = model.encode_image(images)
                text_features = model.encode_text(texts)
                
                logits_per_image, logits_per_text = model(images, texts) # Forward pass
                
                ground_truth_img = torch.arange(logits_per_image.shape[0], dtype=torch.long, device=device)
                # Ground truth labels for text
                ground_truth_txt = -1 * torch.ones(len(classes), dtype=torch.long, device=device)
                for idx, class_label in enumerate(classes):
                    if class_label in label_ids:
                        ground_truth_txt[idx] = (label_ids == class_label).nonzero(as_tuple=True)[0].item()

                print(f"logits_per_image shape: {logits_per_image.shape}")
                print(f"logits_per_text shape: {logits_per_text.shape}")
                print(f"ground_truth shape: {ground_truth.shape}")
                print(f"ground_truth_img: {ground_truth_img}")
                print(f"ground_truth_txt: {ground_truth_txt}")
                
                # Compute loss
                img_loss = loss_img(logits_per_image, ground_truth_img)
                txt_loss = loss_txt(logits_per_text, ground_truth_txt)
                total_loss = (img_loss + txt_loss) / 2
                epoch_test_loss += total_loss
            
            # Normalize features
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            
            assert torch.equal(logits_per_image.T, logits_per_text), "Logits are not equal"
            
            # Compute cosine similarity
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            
            # Compute top accuracy
            acc_top1 = torchmetrics.functional.accuracy(similarity, label_ids, task="multiclass", num_classes=num_classes)
            acc_top3 = torchmetrics.functional.accuracy(similarity, label_ids, task="multiclass", num_classes=num_classes, top_k=3)
            
            acc_top1_list.append(acc_top1)
            acc_top3_list.append(acc_top3)

        epoch_test_loss /= num_batches_test # Average loss per epoch
        writer.add_scalar("Loss/test", epoch_test_loss, epoch) # Log loss to TensorBoard

        print(f"Epoch {epoch} train loss: {epoch_train_loss / num_batches_train}")
        print(f"Epoch {epoch} test loss: {epoch_test_loss / num_batches_test}")
        
        # Compute mean top3 and top1 accuracy
        mean_top3_accuracy = torch.stack(acc_top3_list).mean().cpu().numpy()
        print(f"Mean Top 3 Accuracy: {mean_top3_accuracy*100:.2f}%")
        writer.add_scalar("Test Accuracy/Top3", mean_top3_accuracy, epoch)
        mean_top1_accuracy = torch.stack(acc_top1_list).mean().cpu().numpy()
        print(f"Mean Top 1 Accuracy: {mean_top1_accuracy*100:.2f}%")
        writer.add_scalar("Test Accuracy/Top1", mean_top1_accuracy, epoch)
        
    writer.flush() # Make sure all pending writes are completed
    writer.close()

In [47]:
finetune()

Train dataset size: 74
Test dataset size: 19
Epoch 1/10


  0%|          | 0/2.3125 [00:00<?, ?it/s]

images shape: torch.Size([11, 3, 224, 224])
label_ids shape: torch.Size([11])
label_ids: tensor([ 7,  6,  8,  9, 10,  1,  4,  5,  3,  0,  2])
text shape: torch.Size([11, 77])
logits_per_image shape: torch.Size([11, 11])
logits_per_text shape: torch.Size([11, 11])
ground_truth shape: torch.Size([11])
ground_truth: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])


 43%|████▎     | 1/2.3125 [00:05<00:06,  5.16s/it]

images shape: torch.Size([11, 3, 224, 224])
label_ids shape: torch.Size([11])
label_ids: tensor([ 8,  1,  5,  2,  0,  6,  7,  9,  3, 10,  4])
text shape: torch.Size([11, 77])
logits_per_image shape: torch.Size([11, 11])
logits_per_text shape: torch.Size([11, 11])
ground_truth shape: torch.Size([11])
ground_truth: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])


...

images shape: torch.Size([7, 3, 224, 224])
label_ids shape: torch.Size([7])
label_ids: tensor([4, 7, 0, 8, 3, 5, 6])
texts shape: torch.Size([11, 77])


168%|██████████| 1/0.59375 [00:03<-1:59:59,  3.74s/it]

logits_per_image shape: torch.Size([7, 11])
logits_per_text shape: torch.Size([11, 7])
ground_truth shape: torch.Size([11])
ground_truth_img: tensor([0, 1, 2, 3, 4, 5, 6])
ground_truth_txt: tensor([ 2, -1, -1,  4,  0,  5,  6,  1,  3, -1, -1])
images shape: torch.Size([7, 3, 224, 224])
label_ids shape: torch.Size([7])
label_ids: tensor([8, 0, 7, 5, 3, 6, 4])
texts shape: torch.Size([11, 77])


2it [00:06,  3.22s/it]                                

logits_per_image shape: torch.Size([7, 11])
logits_per_text shape: torch.Size([11, 7])
ground_truth shape: torch.Size([11])
ground_truth_img: tensor([0, 1, 2, 3, 4, 5, 6])
ground_truth_txt: tensor([ 1, -1, -1,  4,  6,  3,  5,  2,  0, -1, -1])
Epoch 9 train loss: 0.8230394124984741
Epoch 9 test loss: 15.124711036682129
Mean Top 3 Accuracy: 100.00%
Mean Top 1 Accuracy: 85.71%


In [53]:
# Model inference

device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load model
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

# Load the fine-tuned weights
checkpoint_path = Path("model_checkpoints/epoch_9.pt")
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# Load image
image_path = os.path.join("test_imgs/test_group.jpg")
image = Image.open(image_path)

# Preprocess image
image = preprocess(image).unsqueeze(0).to(device)

# Generate text prompt
keywords = ["chopping-board", "glass-bowl-large", "glass-bowl-medium", "glass-bowl-small", "group_step", "oven-dish", "oven-tray", "pan", "pot-one-handle", "pot-two-handles-medium", "pot-two-handles-small"]
text_prompts = [f"A photo of a {keyword}" for keyword in keywords]
tokenized_text = clip.tokenize(text_prompts).to(device)

# Generate features
with torch.no_grad():
    image_features = model.encode_image(image)
    image_features /= image_features.norm(dim=-1, keepdim=True) # Normalize features
    
    text_features = model.encode_text(tokenized_text)
    text_features /= text_features.norm(dim=-1, keepdim=True) # Normalize features

# Compute cosine similarity
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Get predicted keyword
predicted_prob, predicted_keyword_idx = similarity.topk(1, dim=-1)

# Print prediction
predicted_keyword = keywords[predicted_keyword_idx.item()]
print(f"Predicted keyword: {predicted_keyword} with probability {predicted_prob.item() * 100:.2f}%")

Predicted keyword: chopping-board with probability 44.89%
